In [31]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import numpy as np
import torch 
import torchvision
from PIL import Image
import json
import datetime
from torchvision.transforms import transforms 
from torchvision.utils import make_grid
import torch.nn as nn
import time
import copy
import torchvision.models as models
resnet = models.resnet18(pretrained=True)


In [32]:
resnet.fc = nn.Linear(512, 128)

print(resnet)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [33]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

In [34]:

class CPC(nn.Module):

    def __init__(self,
                 z_dim=128,
                 cpc_batch=3,
                 batch_size=1,
                 loader=None,
                 gpu_id=0,
                 rescale=1000.0,
                 ):
        super(CPC, self).__init__()

        self.z_dim = z_dim
        self.cpc_batch = cpc_batch
        self.batch_size = batch_size
        self.loader = loader
        self.rescale = rescale
        self._check_inputs()

        self.z_dim = z_dim

        self.device = torch.device(
            "cuda:"+str(gpu_id)) if torch.cuda.is_available() else torch.device("cpu")

        self.W = nn.Parameter(torch.rand(z_dim, z_dim), requires_grad=True)

    def _check_inputs(self):
        assert self.z_dim is not None
        assert self.cpc_batch is not None
        assert self.batch_size is not None
        assert self.loader is not None

    def encode(self, x):
        return x

    def loss(self, x, x_next):
        # Same as density
        assert x_next.size(0) == x.size(0)
        z = self.encode(x)
        z_next = self.encode(x_next)
        # z = z.unsqueeze(2)  # bs x z_dim x 1
        #z_next = z_next.unsqueeze(2)
        w = self.W

        w = w.expand(x.shape[0], w.shape[0], w.shape[1])

        f_out = torch.bmm(torch.bmm(z_next, w), z.permute(0, 2, 1))
        f_out = f_out.squeeze()
        f_out /= self.rescale

        f_pos = f_out[:, :1]

        loss = log_sum_exp(f_out) - torch.log(torch.exp(f_pos))
        #print('neg',log_sum_exp(f_out))
        #print('pos',torch.log(torch.exp(f_pos)))

        #assert False 
        return torch.mean(loss), None

    def forward(self, z1, z2):
        z1 = z1.view(128, 1)
        z2 = z2.view(-1, 128)
        return torch.exp(torch.matmul(torch.matmul(z2, self.W), z1).squeeze(0)/self.rescale)


In [38]:
from roofnet.utils.data import ImageDataset, TripletBuildingSampler
from torchvision.transforms import transforms 

gpu_id = 0 
device = torch.device("cuda:"+str(gpu_id)) if torch.cuda.is_available() else torch.device("cpu")

data_file_path = '/home/misha/downloads/roofnet/data/small_dataset.npy'
data = ImageDataset(data_file_path, 
                    transform=transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize(
                                    (0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                            ]))

sampler = TripletBuildingSampler(data,batch_size=1)
loader = torch.utils.data.DataLoader(data, sampler=sampler,batch_size=3)
loaders = {'train':loader,'val':loader}
cpc = CPC(z_dim=128,
          cpc_batch=3,
          batch_size=1,
          gpu_id=gpu_id,
          loader=loader).to(device)

optimizer = torch.optim.Adam(cpc.parameters(), lr=3e-4)

model = nn.Sequential(resnet, cpc)


Loading data
Done loading data


In [50]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels,meta in dataloaders[phase]:
            
                print(inputs.shape,labels,meta)
                assert False 
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [51]:
train_model(model,loaders,cpc.loss,optimizer)

Epoch 0/24
----------
torch.Size([3, 3, 32, 64]) tensor([2015, 2013, 2018]) {'transition_year': tensor([2019, 2019, 2019]), 'address': ['5431 W 104th St, Los Angeles, CA 90045, USA', '5431 W 104th St, Los Angeles, CA 90045, USA', '5431 W 104th St, Los Angeles, CA 90045, USA']}


AssertionError: 